In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import inception_v3
from PIL import ImageFile
import tqdm
from torch.cuda.amp import autocast, GradScaler

# ✅ Handle truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# ✅ Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True  # ✅ Faster training

# ✅ Paths to Dataset
train_dir = "/home/vu-lab03-pc17/Desktop/Raghu -28/deep_learning/final_dataset/New/data_split_new/train"
val_dir = "/home/vu-lab03-pc17/Desktop/Raghu -28/deep_learning/final_dataset/New/data_split_new/val"
test_dir = "/home/vu-lab03-pc17/Desktop/Raghu -28/deep_learning/final_dataset/New/data_split_new/test"

# ✅ Transformations (For Inception v3)
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Inception v3 expects 299x299 input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ✅ Load Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# ✅ Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# ✅ Model (Using Inception v3)
model = inception_v3(weights="IMAGENET1K_V1", aux_logits=True)  # ✅ Keep aux_logits=True
model.fc = nn.Linear(model.fc.in_features, 1)  # Binary classification
model = model.to(device)

# ✅ Loss & Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# ✅ Mixed Precision
scaler = GradScaler()

# ✅ Training Function
def train(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    train_losses, val_losses = [], []

    for epoch in range(epochs):
        print(f"Epoch No: {epoch + 1}")
        model.train()
        total_loss = 0

        for inputs, labels in tqdm.tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device, dtype=torch.float32).unsqueeze(1)
            
            optimizer.zero_grad()
            with autocast():
                outputs = model(inputs)
                if isinstance(outputs, tuple):  # ✅ Ignore auxiliary logits
                    outputs = outputs[0]
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # ✅ VALIDATION
        model.eval()
        total_loss = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device, dtype=torch.float32).unsqueeze(1)
                with autocast():
                    outputs = model(inputs)
                    if isinstance(outputs, tuple):  # ✅ Ignore auxiliary logits
                        outputs = outputs[0]
                    loss = criterion(outputs, labels)
                total_loss += loss.item()

        avg_val_loss = total_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        print(f'Epoch {epoch + 1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

    return train_losses, val_losses

# ✅ Train Model
train_losses, val_losses = train(model, train_loader, val_loader, criterion, optimizer, epochs=10)

# ✅ Save Model
torch.save(model.state_dict(), "binary_classification_inception1.pth")
print("Training Completed & Model Saved! 🚀")


/tmp/ipykernel_102935/110841570.py:52: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Epoch No: 1


/tmp/ipykernel_102935/110841570.py:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.    | 0/4400 [00:00<?, ?it/s]
  with autocast():
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4400/4400 [03:33<00:00, 20.65it/s]
/tmp/ipykernel_102935/110841570.py:88: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1, Train Loss: 0.1826, Val Loss: 0.2547
Epoch No: 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4400/4400 [02:01<00:00, 36.25it/s]


...

Epoch 8, Train Loss: 0.0524, Val Loss: 0.0765
Epoch No: 9


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4400/4400 [02:02<00:00, 35.93it/s]


Epoch 9, Train Loss: 0.0497, Val Loss: 0.0496
Epoch No: 10


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4400/4400 [02:02<00:00, 35.92it/s]


Epoch 10, Train Loss: 0.0492, Val Loss: 0.0480
Training Completed & Model Saved! 🚀


In [3]:
# ✅ Function to Evaluate Model on Test Data
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device, dtype=torch.float32).unsqueeze(1)
            outputs = model(inputs)
            if isinstance(outputs, tuple):  # ✅ Ignore auxiliary logits
                outputs = outputs[0]
            predictions = (torch.sigmoid(outputs) > 0.5).float()  # Convert logits to binary predictions
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total * 100
    return accuracy

# ✅ Load Saved Model
model.load_state_dict(torch.load("binary_classification_inception1.pth"))
model.to(device)

# ✅ Compute Test Accuracy
test_accuracy = test_model(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 98.33%


In [1]:
import plotly.graph_objects as go

# ✅ Function to Plot Loss Curves
def plot_loss_curve(train_losses, val_losses):
    epochs = list(range(1, len(train_losses) + 1))
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=epochs, y=train_losses, mode='lines+markers', name='Train Loss'))
    fig.add_trace(go.Scatter(x=epochs, y=val_losses, mode='lines+markers', name='Validation Loss'))
    
    fig.update_layout(
        title="Training & Validation Loss Curve",
        xaxis_title="Epoch",
        yaxis_title="Loss",
        template="plotly_dark",
        legend=dict(x=0, y=1)
    )
    
    fig.show()

# ✅ Plot Loss Graph
plot_loss_curve(train_losses, val_losses)


NameError: name 'train_losses' is not defined

In [8]:
#!pip install plotly

In [14]:
import torch
import torchvision.transforms as transforms
from torchvision.models import inception_v3
from PIL import Image

# ✅ Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load the Trained Model
model = inception_v3(weights=None, aux_logits=True)  # ✅ Aux logits must match training
model.fc = torch.nn.Linear(model.fc.in_features, 1)  # Binary classification
model.load_state_dict(torch.load("binary_classification_inception.pth", map_location=device))
model.to(device)
model.eval()

# ✅ Image Preprocessing (Same as Training)
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Inception v3 expects 299x299 input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ✅ Function to Predict
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")  # Open and Convert to RGB
    image = transform(image).unsqueeze(0).to(device)  # Apply Transforms & Add Batch Dimension

    with torch.no_grad():
        output = model(image)
        if isinstance(output, tuple):  # Ignore auxiliary logits
            output = output[0]
        prediction = torch.sigmoid(output).item()  # Convert Logits to Probability

    # ✅ Convert to Class Label (Assuming 0 = Class A, 1 = Class B)
    label = "real" if prediction > 0.5 else "fake"
    confidence = prediction if prediction > 0.5 else 1 - prediction

    print(f"Prediction: {label} (Confidence: {confidence:.2f})")

# ✅ Test with an Image
image_path = "/home/vu-lab03-pc17/Downloads/frame_3.jpg"  # Change to your image path
predict_image(image_path)


Prediction: real (Confidence: 0.87)
